# Funnel Analysis 

## Overview
We don't have real longitudinal data, but we have a way of creating a proxy for the same sort of insight. There are questions related to: 
* longevity: how long been using mobile money
* frequency: how often uses mobile money
* recency: how recently used mobile money

Intuitively, we can imagine cases where individuals learned about mobile money a long time ago (exposure), signed up (enrollment), used the services regularly (engagement) and continue to do so (retention).

In this funnel analysis, we see these as progressive steps (e.g.. "out of those who have heard of mobile money, how many have an account?"... "Out of those that have a mobile money account, how many use mobile money services regularly"). We treat each step as a binary classification problem, and try to find those features and demographics that are predictive of progressing "to the next stage" of the funnel.

## Data Sources

The data source is the Financial Inclusion Insights Data (<a href="http://finclusion.org/about/">http://finclusion.org/about/</a>). 
 
  * We consider responses from Uganda, Kenya, and Tanzania, the countries of interest for this project
  * We consider only "wave 3" questions that relate to these concepts.

The source data is in Microsoft XLSX format. Preprocessing steps include format conversion from XLSX to CSV, merging columns for those survey responses that are identical across all countries, and renaming columns to a human readable format. For example: 

`'MM9' --> 'how_long_using_mobile_money'`


## Survey questions

We have selected Wave 3 questions that relate to the concepts of interest. We have also broken down survey responses to separate usage of mobile money into "basic" and more "sophisticated" categories, where the former ("basic services") is mostly transferring money, and the later ("sophisticated services") includes activities like long-term savings and insurance.

Concepts: 
 Exposure
 Enrollment
 Usage of basic financial services
 Usage of sophisticated financial services
 Recency
 Longevity
 Frequency
 (Retention)
 
The specific questions and responses considered to address each of these concepts are detailed below in the corresponding sections. 

## Funnel Analysis

[Describe the funnel path and flow] 

[Describe what attributes of respondants are examined at each step] 

## Results and further information

[available in the project deliverable report] 


# --------------------

In [3]:
%matplotlib inline

In [4]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn

/home/aman/anaconda/lib/python2.7/site-packages/pandas/computation/__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)
/home/aman/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [5]:
"""
## Load Data

 
"""
input_data_dir = 'Data/'
# os.listdir(input_data_dir)
df = pd.DataFrame.from_csv(
    os.path.join(input_data_dir, 'merged_fii_data_wave3_20160915.csv'),
)

['fii_csv',
 'merged_fii_data_wave3_20160915.csv',
 'FII',
 'merged_fii_data_wave3_20160910.csv',
 'FIndex',
 'findex_csv']

## Funnel step: Exposure

Survey Questions: 

 * MM1. Have you ever heard of something called Mobile Money? 


In [ ]:
Wave 3 questions (Uganda, Kenya, Tanzania) questions that relate to these concepts.

Exposure:
MM1. Have you ever heard of something called Mobile Money? 

Enrollment:
MM4. Have you ever used this mobile money service for any financial activity?
MM6_n Do you have a registered account (account registered in your name) with [any] mobile money service?
+ no answer for MM12 What is the main reason you have never used mobile money services?

Usage of Basic Financial Services 
MM15. Have you ever used a mobile money account to do [any of] the following [basic services]?
[1-2] deposit, withdraw
[3-10] pay bills
[11 - 12] Send, Receive money

Usage of sophisticated financial services
MM15. Have you ever used a mobile money account to do [any of] the following [sophisticated financial services]?
  [16] Insurance
  [18, 19, ] long term savings
  [20, ] emergency funds
  [17] large transactions, regular transactions (like loans, mortgage)

Recency:
User has used mobile financial services in the recent path
MM5_n. Apart from today, when was the last time you conducted any financial activity with [any] mobile money service?
MM8_n. Apart from today, when was the last time you conducted any financial activity using these registered accounts?

Longevity
learned about / signed up for financial services some time ago
MM9 For how long have you been using mobile money services?

Frequency
Uses financial services regularly / frequently
MM17. How often do you use mobile money for [any] activity?
[could break this down into basic and sophisticated financial activities as well] 

Retention

